### 분류 평가
 - 정확도만으로 불균형한 레이블 데이터 세트에서 평가지표로 사용하기에는 부적합
 - 정확도가 가지는 분류 평가 지표로의 한계점을 극복하기 위해 여러 가지 분류 지표와 함께 적용해야 함
 
### Confusion Matrix(혼동행렬, 오차행렬)
 - 이진 분류에서 성능 지표로 잘 활용되는 오차행렬(혼동행렬)은 학습된 분류 모델이 예측을 수행하면 얼마나 혼동될 수 있는지도 함께 보여주는 지표임
 - 이진 분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측오류가 발생하고 있는지를 함께 보여줌.

### 평가 지표

##### 모델에서 예측을 참 으로 했냐  거짓으로 했냐가 뒤에 붙은 N ,P
##### 실제값이랑 비교했을때 실제랑 같으면 앞에 True, 다르면 False
 - TP, FP, FN, TP는 예측 클래스와 실제 클래스의 Positive 결정 값과 Negative 결정 값의 결합에 따라 결정
 - 앞문자 True/False는 예측값과 실제값이 같은가/틀린가를 의미하고 뒤 문자 Negative/Positive는 예측 결과 값이 부정/긍정을 의미
 - TN는 예측값을 Negative값 0으로 예측했고 실제값 역시 Negative 값0
 - FP는 예측값을 Positive 값 1로 예측했고 실제값은 Negative 값 0
 - FN은 예측값을 Negative 값 0으로 예측했고 실제값은 Positive 값 1
 - TP는 예측값을 Positive 값 1로 예측했고 실제값 역시 Positive 값 1
 - 정확도(accuracy) = (TP + TN) / (TP+TN+FP+FN)
 - 정밀도 = TP / (TP+FP) : P로 예측한 것 중에서 실제도 P
 - 재현율 = TP / (TP+FN) : 실제 P인 것 중에서 예측도 P
 - F1 = 2 * (정밀도 * 재현율) / (정밀도 + 재현율) : 정밀도와 재현율이 어느 한쪽으로 치우치지 않는 수치를 나타낼 때 높아짐
 - 정밀도와 재현율은 Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가 지표
 - 재현율이 중요 지표인 경우는 실제 Positive 양성 데이터를 Negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우 (ex.보험사기)
 - 정밀도가 더 중요한 지표인 사례는 스팸 메일 여부를 판단하는 경우로 스팸 메일이 아닌데 스팸 메일로 분류해서 업무 차질 발생

In [1]:
TN = 143
FP = 15
FN = 37
TP = 67
accuracy = ((TP+TN) / (TP+TN+FP+FN))
precision = TP / (TP+FP)
recall = TP / (TP+FN)
F1 = 2 * (precision*recall) / (precision+recall)

print('accuracy : ',accuracy)
print('precision : ',precision)
print('recall : ',recall)
print('F1 : ',F1)

accuracy :  0.8015267175572519
precision :  0.8170731707317073
recall :  0.6442307692307693
F1 :  0.7204301075268817


In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 15)
titanic_df=pd.read_csv('titanic3.csv')

from sklearn.preprocessing import LabelEncoder
# Null 처리 함수
def fillna(df):
    df['age'].fillna(df['age'].mean(), inplace=True)
    df['cabin'].fillna('N', inplace=True)
    df['fare'].fillna(df['fare'].mean(), inplace=True)
    df['embarked'].fillna('N', inplace=True)
    return df

def drop_features(df):
    df.drop(['home.dest','boat','body','name','ticket'], axis=1, inplace=True)
    return df

def format_features(df):
    df['cabin'] = df['cabin'].str[:1]
    features = ['cabin','sex','embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

t_df = transform_features(titanic_df)
t_df.to_pickle('t_df.pkl')

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 분리
X = t_df[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'cabin', 'embarked']]
y = t_df['survived']

# 독립변수 정규화(평균 0, 분산1인 표준정규분포)
X = preprocessing.StandardScaler().fit(X).transform(X) # StandardScaler : 표준정규분포

# 학습용 데이터와 평가용 데이터를 8:2로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# SVM
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.metrics import classification_report
from sklearn import svm
# 벡터 공간으로 매핑하는 함수를 커널이라고 함
# kernal = 'rbf' 옵션으로 RBF(Radial Basis Function) 함수를 적용

svm_model = svm.SVC(kernel='rbf', random_state=0) # random_state=0 으로 정해서 예측치 변동이 생기지 않게 한다.
# kernel :  구분하는 선을 기준으로 선과 데이터의 거리가 margin이다. 
# 하지만, 하나의 선으로 분류할 수 있는 경우는 거의 없기 때문에 1차원을 2차원으로 만들고, 2차원을 3차원으로 만든다.
# 선을 차원으로 바꿔주는 것이 kernel.

svm_model.fit(X_train, y_train)
s_pred = svm_model.predict(X_test)

s_accuracy = accuracy_score(y_test, s_pred)
print('s 예측 정확도 :', s_accuracy)

s 예측 정확도 : 0.8015267175572519


### 정밀도 및 재현율 활용시 유의사항
 - 정밀도와 재현율 성능 수치는 어느 한쪽만 참조하면 극단적인 수치 조작이 가능
 - 정밀도 100%가 되는 방법 : 확실한 기준이 되는 경우만 Positive로 예측하고 나머지는 모두 Negative로 예측 전체 환자 1000명 중 확실한 Positive징후만 가진 환자는 단1명 이라고 하면, 이 한명만 P로 예측하고 나머지는 모두 N으로 예측 FP는 0, TP는 1이 되며 정밀도 (TP/(TP+FP)는 1/(1+0)=1
 - 재현율이 100%가 되는 방법 : 모든 환자를 Positive로 예측 1000명의 환자 중 실제 양성인 사람이 30명 정도라도 TN이 수치에 포함되지 않고 FN은 0 이므로 재현율 (TP/(TP+FN)은 30/(30+0) =1
 - 분류가 정밀도, 재현율 중 하나에 상대적인 중요도를 부여할 수 있지만 하나만 강조해서는 안됨
 - 암 예측 모델에서 재현율을 높인다고 주로 양성만 판정한다면 환자의 부담과 불평이 커지게 됨

In [3]:
def accuracy(TP,TN,FP,FN):
    accuracy=((TP+TN) / (TP+TN+FP+FN))
    return accuracy

def precision(TP,FP):
    precision=TP / (TP+FP)
    return precision

def recall(TP,FN):
    recall = TP / (TP+FN)
    return recall

def F1(precision,recall):
    F1=2* (precision*recall) / (precision+recall)
    return F1

# confusion matrix
confusion = confusion_matrix(y_test, s_pred)
print(confusion)
#[[143  15]  생존자 기준일 때, 143이 TN, 15가 FP에 해당
# [ 37  67]]  37이 FN에, 67이 TP에 해당

#생존자 기준
TP = 67
TN = 143
FP = 15
FN = 37

acc=accuracy(TP,TN,FP,FN) # 생존률과 사망률 둘다 비슷하다
pre=precision(TP,FP) # 정밀도
recall=recall(TP,FN) #재현율
F1=F1(pre,recall)
print(acc, pre, recall, F1)

svm_report = classification_report(y_test, s_pred)
print(svm_report)

[[143  15]
 [ 37  67]]
0.8015267175572519 0.8170731707317073 0.6442307692307693 0.7204301075268817
              precision    recall  f1-score   support

           0       0.79      0.91      0.85       158
           1       0.82      0.64      0.72       104

    accuracy                           0.80       262
   macro avg       0.81      0.77      0.78       262
weighted avg       0.80      0.80      0.80       262



In [4]:
# 사망자 기준
TN = 67
TP = 143
FN = 15
FP = 37

acc=accuracy(TP,TN,FP,FN)
pre=precision(TP,FP)
recall= TP / (TP+FN)
F1= 2 * (pre*recall) / (pre+recall)
print(acc, pre, recall, F1)

0.8015267175572519 0.7944444444444444 0.9050632911392406 0.8461538461538461


In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_eval(y_test, s_pred):
    confusion = confusion_matrix(y_test, s_pred)
    accuracy = accuracy_score(y_test, s_pred)
    precision = precision_score(y_test, s_pred)
    recall = recall_score(y_test, s_pred)
    f1 = f1_score(y_test, s_pred)
    
    print('오차행렬')
    print(confusion)
    print()
    print('정확도 : {0:.4f}, 정밀도  {1:.4f}, 재현율 : {2:.4f}, F1:{3:.4f}'.format(accuracy, precision, recall, f1))

get_clf_eval(y_test, s_pred)

오차행렬
[[143  15]
 [ 37  67]]

정확도 : 0.8015, 정밀도  0.8171, 재현율 : 0.6442, F1:0.7204


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

t_df = pd.read_pickle('t_df.pkl')

y_df = t_df.survived
X_df = t_df.drop('survived', axis=1)

# 학습용 평가용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=11)

# 분류기 객체 생성
lr_clf = LogisticRegression(random_state=11)
# dt_clf = DecisionTreeClassifier(random_state=11)
# rf_clf = RandomForestClassifier(random_state=11)

# GridSearchCV : 파라미터를 통해 성능을 튜닝. (타이타닉 생존률을 해결하기 위해 필요한 작업)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

parameters =  parameters = {'penalty':['l2','l1'],'C':[0.01,0.1,1,1,5,10]}

grid_lrlf = GridSearchCV(lr_clf, param_grid=parameters, scoring='accuracy', cv=5, refit=True) 

display(grid_lrlf)
grid_lrlf.fit(X_train, y_train) 

print('GridSearchCV 최적 하이퍼 파라미터:', grid_lrlf.best_params_) 

print('GridSearchCV 최고 정확도:', grid_lrlf.best_score_) 

best_lrlf = grid_lrlf.best_estimator_
display(best_lrlf)
dpredictions = best_lrlf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)

def get_clf_eval(y_test, dpredictions):
    confusion = confusion_matrix(y_test, dpredictions)
    accuracy = accuracy_score(y_test, dpredictions)
    precision = precision_score(y_test, dpredictions)
    recall = recall_score(y_test, dpredictions)
    f1 = f1_score(y_test, dpredictions)
    
    print('오차행렬')
    print(confusion)
    print()
    print('정확도 : {0:.4f}, 정밀도  {1:.4f}, 재현율 : {2:.4f}, F1:{3:.4f}'.format(accuracy, precision, recall, f1))

get_clf_eval(y_test, dpredictions)

print('Logistic Regression GSCV 예측 정확도 : ', accuracy) 

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=11, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1, 1, 5, 10],
                         'penalty': ['l2', 'l1']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

GridSearchCV 최적 하이퍼 파라미터: {'C': 1, 'penalty': 'l2'}
GridSearchCV 최고 정확도: 0.7841330599225337


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=11, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

오차행렬
[[139  17]
 [ 30  76]]

정확도 : 0.8206, 정밀도  0.8172, 재현율 : 0.7170, F1:0.7638
Logistic Regression GSCV 예측 정확도 :  0.8206106870229007
